# AI Career Advisor Agent - Revamped with LoopAgent

This notebook implements an interactive career advisor using:
- **LoopAgent** for interactive intake (gathering user information)
- **SequentialAgent** for the main pipeline (research + mentoring)
- **Memory** for cross-session context

## Architecture
```
User Query
    ↓
Intake Loop (asks questions until profile complete)
    ↓
Research Agent (searches for career transition info)
    ↓
Mentor Agent (creates personalized action plan)
```

In [ ]:
# Import dependencies
from dotenv import load_dotenv
import os

load_dotenv()

from google.adk.agents import LlmAgent, SequentialAgent, LoopAgent
from google.adk.runners import Runner
from google.adk.models.google_llm import Gemini
from google.adk.tools import google_search, preload_memory, FunctionTool
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService

print("✅ ADK components imported successfully.")

In [ ]:
# Setup API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
else:
    print("🔑 Authentication Error: 'GOOGLE_API_KEY' not found in .env file.")

In [ ]:
# Configuration
APP_NAME = "career_advisor"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

# Retry configuration
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("✅ Configuration loaded.")

In [ ]:
# Helper function for running sessions
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
    show_all_agents: bool = False,
):
    """Run queries in a session and display agent responses."""
    print(f"\n{'='*60}")
    print(f"Session: {session_name}")
    print(f"{'='*60}")

    app_name = runner_instance.app_name

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
        print("✅ New session created")
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
        print("✅ Existing session retrieved")

    if user_queries:
        if isinstance(user_queries, str):
            user_queries = [user_queries]

        for query in user_queries:
            print(f"\n👤 User: {query}")
            print(f"{'-'*60}")

            query_content = types.Content(role="user", parts=[types.Part(text=query)])

            responses = []

            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query_content
            ):
                if event.content and event.content.parts:
                    text = event.content.parts[0].text
                    if text and text != "None":
                        agent_name = getattr(event, 'author', 'Agent')

                        if show_all_agents:
                            print(f"\n🤖 {agent_name}:")
                            print(text)
                        else:
                            responses.append({'agent': agent_name, 'text': text})

            # If not showing all, print only final response
            if not show_all_agents and responses:
                last = responses[-1]
                print(f"\n🤖 {last['agent']}:")
                print(last['text'])
    else:
        print("⚠️  No queries provided!")

print("✅ Helper function defined.")

In [ ]:
# Memory callback
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    try:
        agent_name = callback_context._invocation_context.agent.name
        session = callback_context._invocation_context.session

        await callback_context._invocation_context.memory_service.add_session_to_memory(session)

        print(f"💾 Saved {agent_name} output to memory")
    except Exception as e:
        print(f"⚠️  Warning: Failed to save to memory: {e}")

print("✅ Callback created.")

## Tool Functions

In [ ]:
# Validation function
def validate_user_profile(
    current_role: str,
    years_experience: int,
    target_role: str,
    time_commitment_hours: int = 10
) -> dict:
    """
    Validates that all required user profile information has been collected.
    
    Args:
        current_role: The user's current job role
        years_experience: Years of experience in current role (0-50)
        target_role: The role the user wants to transition to
        time_commitment_hours: Hours per week available for learning (1-168)
    
    Returns:
        Dictionary with status and message
    """
    # Validate inputs
    if not current_role or len(current_role.strip()) < 2:
        return {"status": "incomplete", "message": "Current role is missing or invalid"}
    
    if not target_role or len(target_role.strip()) < 2:
        return {"status": "incomplete", "message": "Target role is missing or invalid"}
    
    if not isinstance(years_experience, int) or years_experience < 0 or years_experience > 50:
        return {"status": "invalid", "message": "Years of experience must be a number between 0 and 50"}
    
    if not isinstance(time_commitment_hours, int) or time_commitment_hours < 1 or time_commitment_hours > 168:
        return {"status": "invalid", "message": "Time commitment must be between 1 and 168 hours per week"}
    
    # All validations passed
    return {
        "status": "complete",
        "message": f"✅ Profile complete! Ready to research transition from {current_role} to {target_role}.",
        "profile": {
            "current_role": current_role,
            "years_experience": years_experience,
            "target_role": target_role,
            "time_commitment_hours": time_commitment_hours
        }
    }

# Exit function for loop
def exit_intake_loop():
    """Exit the intake loop when profile is complete."""
    return {"status": "profile_complete", "message": "Proceeding to research and mentoring..."}

print("✅ Validation and exit functions created.")

## Agent Definitions

In [ ]:
# Question Agent - Asks one question at a time
question_agent = LlmAgent(
    name="question_agent",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    instruction="""
    You are a friendly Question Agent gathering information for career transition planning.
    
    Required information (4 items):
    1. current_role - User's current job/position
    2. years_experience - Years in current role (must be a NUMBER)
    3. target_role - Role they want to transition to
    4. time_commitment_hours - Hours per week for learning (must be a NUMBER, default: 10)
    
    PROCESS:
    - Review the conversation history to see what's already been provided
    - Check preload_memory for any past information
    - If something is missing, ask ONE friendly question for the next missing piece
    - If you receive an answer, acknowledge it briefly and naturally
    - DO NOT call any validation tools - that's the next agent's job
    
    IMPORTANT:
    - Ask questions conversationally, not like a form
    - Be warm and encouraging
    - Only ask ONE question at a time
    - If the user provides multiple pieces of info at once, acknowledge all of them
    """,
    tools=[preload_memory],
    output_key="conversation_state",
)

print("✅ Question agent created.")

In [ ]:
# Validator Agent - Checks if we have all info and exits loop if complete
validator_agent = LlmAgent(
    name="validator_agent",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    instruction="""
    You are a Validator Agent. Review the conversation to determine if all required profile information has been collected.
    
    Required information:
    1. current_role (string)
    2. years_experience (integer/number)
    3. target_role (string)
    4. time_commitment_hours (integer/number)
    
    CRITICAL DECISION LOGIC:
    
    IF you have all 4 pieces of information:
        1. Call validate_user_profile() with the gathered data
        2. If validation returns "complete":
           - Call exit_intake_loop() IMMEDIATELY
           - Output the validated profile summary
        3. If validation returns "incomplete" or "invalid":
           - Do nothing, let the loop continue
    
    IF you're missing any information:
        - Do NOT call any functions
        - Just output a brief acknowledgment
        - The loop will continue and question_agent will ask the next question
    
    IMPORTANT RULES:
    - years_experience and time_commitment_hours MUST be integers (whole numbers)
    - If user says "five years", convert to 5
    - If user says "about 10 hours" or "10-15 hours", use 10
    - You MUST call exit_intake_loop() when profile is complete
    """,
    tools=[FunctionTool(validate_user_profile), FunctionTool(exit_intake_loop)],
    output_key="user_profile",
)

print("✅ Validator agent created.")

In [ ]:
# Intake Loop - Combines question and validator agents
intake_loop = LoopAgent(
    name="IntakeLoop",
    sub_agents=[question_agent, validator_agent],
    max_iterations=10,  # Safety limit to prevent infinite loops
)

print("✅ Intake loop created.")

In [ ]:
# Research Agent
research_agent = LlmAgent(
    name="research_agent",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    instruction="""
    You are a Research Agent specialized in career transitions.
    
    USER PROFILE:
    {user_profile}
    
    Your task:
    1. Use the Google Search tool to research the target career path from the user profile
    2. Focus on: required skills, typical career progression, salary ranges, and job market demand
    3. Search for: online courses, certifications, and learning resources
    4. Look for: success stories of people who made similar transitions
    
    Output format:
    ## Key Skills Required
    [List 5-7 essential skills with brief descriptions]
    
    ## Learning Resources
    [Specific courses, certifications, books, and platforms]
    
    ## Market Outlook
    [Job demand, salary ranges, growth trends with data]
    
    ## Transition Timeline
    [Typical timeframe for this transition based on experience level]
    
    ## Success Stories
    [Brief examples of successful transitions]
    
    Keep your research comprehensive but concise. Focus on actionable information.
    """,
    tools=[google_search, preload_memory],
    output_key="research_summary",
    after_agent_callback=auto_save_to_memory,
)

print("✅ Research agent created.")

In [ ]:
# Mentor Agent
mentor_agent = LlmAgent(
    name="mentor_agent",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    instruction="""
    You are a Career Mentor Agent.
    
    USER PROFILE:
    {user_profile}
    
    RESEARCH FINDINGS:
    {research_summary}
    
    Create a personalized, actionable transition plan tailored to the user's profile.
    
    Structure your plan:
    
    ## Phase 1: Foundation (Months 1-3)
    - Specific skills to learn first (prioritized for their background)
    - Recommended courses/resources with links
    - Time commitment strategy (based on their available hours/week)
    - Quick wins to build confidence
    
    ## Phase 2: Building Portfolio (Months 4-6)
    - Concrete projects to build (relevant to their experience)
    - GitHub repositories to create
    - Communities to join (specific names)
    - Networking strategies
    
    ## Phase 3: Job Search (Months 6-9)
    - Resume updates needed
    - Where to apply (companies, job boards)
    - Interview preparation tips
    - Portfolio presentation strategies
    
    ## Milestones & Checkpoints
    - Monthly goals with measurable outcomes
    - How to measure progress
    - Red flags and when to adjust
    
    ## Leveraging Your Background
    - How to translate their existing skills
    - Unique advantages from their current position
    - How their experience is an asset
    
    Be specific, realistic, and encouraging. Use actual course names and platforms.
    Tone: supportive but practical - acknowledge challenges while emphasizing achievability.
    """,
    output_key="career_advice",
)

print("✅ Mentor agent created.")

## Pipeline Assembly

In [ ]:
# Root Agent - Sequential pipeline with intake loop first
root_agent = SequentialAgent(
    name="CareerAdvisorPipeline",
    sub_agents=[intake_loop, research_agent, mentor_agent],
)

print("✅ Root agent created with 3-stage pipeline:")
print("   1. Intake Loop (question → validate → repeat until complete)")
print("   2. Research Agent (gather career transition info)")
print("   3. Mentor Agent (create personalized action plan)")

In [ ]:
# Initialize services and runner
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

print("✅ Runner initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Single runner for entire pipeline")

## Testing & Usage

In [ ]:
# Test 1: Interactive intake - Start the conversation
# The agent will ask questions one at a time
await run_session(
    runner,
    "I want to change careers",
    session_name="interactive-test-1",
    show_all_agents=True,  # Show all agents to see the loop in action
)

In [ ]:
# Test 2: Continue answering questions in the SAME session
# Answer whatever the agent asked in the previous cell
await run_session(
    runner,
    "Product Manager",  # Example answer to "What's your current role?"
    session_name="interactive-test-1",  # Same session!
    show_all_agents=True,
)

In [ ]:
# Test 3: Continue with more answers
await run_session(
    runner,
    "5 years",  # Years of experience
    session_name="interactive-test-1",
    show_all_agents=True,
)

In [ ]:
# Test 4: Provide remaining info
await run_session(
    runner,
    "Data Scientist and I can dedicate 10 hours per week",  # Multiple pieces at once
    session_name="interactive-test-1",
    show_all_agents=True,
)
# Once all info is gathered, the loop will exit and research+mentor will run automatically!

## Alternative: All-In-One Approach

If the user provides all information upfront, the intake loop will validate and exit immediately, proceeding directly to research and mentoring.

In [ ]:
# Test 5: Provide everything upfront (non-interactive)
await run_session(
    runner,
    "I am a Product Manager with 5 years of experience. I want to transition to Data Science and can dedicate 10 hours per week to learning.",
    session_name="all-in-one-test",
    show_all_agents=False,  # Only show final mentor output
)

## Benefits of This Approach

✅ **Single Runner** - No confusion about which runner to use

✅ **Automatic Flow** - Loop exits when profile is complete, research+mentor run automatically

✅ **Flexible** - Works for both interactive (multi-turn) and direct (all-in-one) approaches

✅ **Validated** - Can't proceed without complete, valid profile data

✅ **Memory-Aware** - Checks past conversations via preload_memory

✅ **Robust** - Max iterations prevents infinite loops